In [5]:
def intToBin(n):
    return ''.join(str(1 & int(n) >> i) for i in range(64)[::-1])


In [6]:
from __future__ import division
from random import Random, random, seed, sample
from datetime import datetime

MRATE = 0.3
CRATE = 1.0

class State(object):
    def __init__(self, listener, crossover_rate = 0.0, mutation_rate = 0.0, parents = None): #todas as classes de indivíduos irão herdar este construtor

        self._fitness = None
        self._probability = None

        self._listener = listener

        self._mutation_rate = mutation_rate
        self._crossover_rate = crossover_rate

        if parents == None:
            self.state = self.initial_state()
        else:
            p1 = parents[0]
            p2 = parents[1]
            self.state = self.crossover(p1, p2)
            self.mutate()
    
    def initial_state(self):
        r = Random()
        return [r.randint(1, 8) for y in range(8)]

    def fitness(self):
        if not self._fitness:
            self._fitness = int(28 - self.collisions())
            self._listener.log()

        return self._fitness

    def collisions(self):
        state = self.state
        horizontal_collisions = sum([(state.count(col) - 1)/2 for col in state])
        diagonal_collisions = 0

        for i, col1 in enumerate(state):
            for j, col2 in enumerate(state):
                dx = abs(col1 - col2)
                dy = abs(i - j)
                if dx == dy and (i != j or col1 != col2):
                    diagonal_collisions += 1
        diagonal_collisions /= 2

        return horizontal_collisions + diagonal_collisions


    def probability(self, population):
        if not self._probability:
            self._probability = self.fitness() / sum([x.fitness() for x in population])

        return self._probability

    def crossover(self, p1, p2):
        r = Random()
        crossover_index = r.randint(0, 8)
        left = p1.state[0:crossover_index]
        right = p2.state[crossover_index:8]
        left.extend(right)
        return left

    def mutate(self):
        r = Random()
        for i in range(len(self.state)):
            if random() < self._mutation_rate:
                self.state[i] = r.randint(1,8)

class PermutationState(State):
    def mutate(self):
        r = Random()
        
        if random() < self._mutation_rate:
            ix1 = r.randint(0,len(self.state)-1)
            ix2 = r.randint(0,len(self.state)-1)
            temp = self.state[ix1]
            self.state[ix1] = self.state[ix2]
            self.state[ix2] = temp


def pickRandomByProbability(populationByProbability):
    value = random()
    current = populationByProbability[0]
    cumulative = current[0]
    i = 0
    while cumulative < value and i < len(populationByProbability)-1:
        i += 1
        current = populationByProbability[i]
        cumulative += current[0]

    return current[1]


class FitnessListerner():
    def __init__(self):
        self._qtd = 0

def generateNextPopulation(listener, population, n, method="roulette", mutation="flip", elitist=False):
    if mutation == "flip":
        stateType = State
    elif mutation == "permutation":
        stateType = PermutationState
    else:
        stateType = State
        

    if method == "roulette":
        return generatePopulationByRoulette(listener, population, n, stateType, elitist=elitist)
    elif method == "tourney":
        return generatePopulationByTourney(listener, population, n, stateType, elitist=elitist)

def generatePopulationByRoulette(listener, population, n, individual, elitist=False):
    populationByFitness = sorted(population, key = lambda x: x.fitness(), reverse = True)
    fitPopulation = populationByFitness[:-2]
    newPopulation = fitPopulation
    minFitness = min([x.fitness() for x in fitPopulation])

    populationByProbability = [(x.probability(fitPopulation), x) for x in fitPopulation]

    while len(newPopulation) < n:
        parent1 = pickRandomByProbability(populationByProbability)
        populationByProbability = [x for x in populationByProbability if x[1] != parent1]
        parent2 = pickRandomByProbability(populationByProbability)
        newIndividual = individual(listener,
                                   crossover_rate = CRATE,
                                   mutation_rate = MRATE,
                                   parents = (parent1, parent2))

        if not elitist or newIndividual.fitness > minFitness:
            newPopulation.append(newIndividual)

    return newPopulation

#função de geração de padrão por torneio
def generatePopulationByTourney(listener, population, n, individual, elitist = False):
    populationByFitness = sorted(population, key = lambda x: x.fitness(), reverse = True)
    fitPopulation = populationByFitness[:-2]
    newPopulation = fitPopulation
    minFitness = min([x.fitness() for x in fitPopulation])

    while len(newPopulation) < n:
        candidates = []
        indices = sample(xrange(0, len(population)-1), 5)
        for i in indices: 
            candidates.append(population[i])

        best = None
        second_best = None

        i = 0
        for c in candidates:
            chosen = False
            if best == None:
                best = c
                chosen = True
            else:
                if c.fitness() > best.fitness():
                    second_best = best
                    best = c
                    chosen = True

            if second_best == None and not chosen:
                second_best = c
                chosen = True
            elif not chosen:
                if c.fitness() > second_best.fitness():
                    second_best = c
                    chosen = True

        newIndividual = individual(listener, 
                                   crossover_rate = CRATE, 
                                   mutation_rate = MRATE, 
                                   parents = (best, second_best))

        if not elitist or newIndividual.fitness() > minFitness:
            newPopulation.append(newIndividual)

    return newPopulation

SyntaxError: unexpected EOF while parsing (<ipython-input-4-e709195c4dd6>, line 1)

In [7]:
from __future__ import division

import state
import time
import csv
import random
import sys


POPULATION_SIZE = 100
MAX_COLLISION = 28
VALID_ARGS = "emg"

class FitnessListener():

    def __init__(self, qtd=0):
        self._qtd = qtd

    def log(self):
        self._qtd += 1

    def retrive_qtd(self):
        return self._qtd

    def reset(self):
        self._qtd = 0

    def copy(self):
        return FitnessListener(self._qtd)

def choose_method(string):
    if "roulette".startswith(string):
        method = "roulette"
    elif "tourney".startswith(string):
        method = "tourney"
    else:
        sys.exit(string + " is not a valid population generation method.")    
    return method

def choose_generations(string):
    try:
        generations = int(string)
    except ValueError:
        sys.exit("Argument " + string + " is not an integer.\nThe argument provided with --generations must be an integer.")
    else:
        return generations



def make_config(arguments):
    elitist = None 
    method = None
    max_generations = None
    mutation_rate = 0.8


    #flag para permitir argumentos "inválidos" se vierem 
    #depois de opções que precisam de argumentos
    next_is_literal_argument = False 

    err = False

    for index, arg in enumerate(arguments[1:]):
        index += 1
        if arg[:2] == "--":
            argstr = arg[2:]
            if argstr == "elitist":
                elitist = True
            elif argstr == "method":
                if len(arguments) > index+1:
                    methodstr = arguments[index+1]
                    method = choose_method(methodstr)
                    next_is_literal_argument = True
                else:
                    sys.exit("--method used, but no method specified for population generation")
            elif argstr == "generations":
                if len(arguments) > index+1:
                    genstr = arguments[index+1]
                    max_generations = choose_generations(genstr)
                    next_is_literal_argument = True
                else:
                    sys.exit("--generations used, but no number of generations specified")
            elif argstr == "mutation":
                mutation_rate = arguments[index+1]
                next_is_literal_argument = True
            else:
                sys.exit("argument " + argstr + " is invalid")
        elif arg[:1] == "-":                    
            argstr = arg[1:]
            err = False

            for c in argstr:
                if c not in VALID_ARGS:
                    print "Unknown command-line argument", c
                    err = True

            if not err:
                if 'e' in argstr:
                    elitist = True
                if 'm' in argstr: 
                    if 'm' not in argstr[:-1] and len(arguments) > index+1:
                        methodstr = arguments[index+1]
                        method = choose_method(methodstr)
                        next_is_literal_argument = True
                    elif 'm' in argstr[:-1]:
                        sys.exit("-m option must be immediately followed by method name")
                    else:
                        sys.exit("-m used, but no method specified for population generation")
                if 'g' in argstr:
                    if  'g' not in argstr[:-1] and len(arguments) > index+1:
                        genstr = arguments[index+1]
                        max_generations = choose_generations(genstr)
                        next_is_literal_argument = True
                    elif 'g' in argstr[:-1]:
                        sys.exit("-g option must be immediately followed by number of generations")
                    else:
                        sys.exit("-g used, but no number of generations specified")
            else:
                sys.exit(1)

        #se o argumento não era válido, levantar um erro
        #se não tivermos a flag de aceitar inválidos
        #levantada
        elif not next_is_literal_argument: 
            print "Unknown command-line argument", arg
            err = True

        #mas caso a flag de aceitar argumento inválido
        #estivesse levantada, precisamos abaixá-la
        else:
            next_is_literal_argument = False

    if err:
        sys.exit(1)
    else:
        return elitist, method, max_generations, mutation_rate

def register_loop(population,generation,results_file):
    maxfitness = max([x.fitness() for x in population])
    print "Generation %d, Max fitness: %d" % (generation, max([x.fitness() for x in population]))
    avgfitness = sum([x.fitness() for x in population])/len(population)
    print "Average fitness:", avgfitness
    results_file.writerow([generation, maxfitness, avgfitness])

if __name__ == '__main__':
    random.seed(time.time())
    generation = 1
    listener = FitnessListener()
    elitist, method, max_generations, mutation_rate = make_config(sys.argv)
    population = [state.State(listener=listener, crossover_rate = 1.0, mutation_rate = mutation_rate) for x in range(POPULATION_SIZE)]

    if elitist == None:
        elitist = False
    if method == None:
        method = "roulette"

    with open('results' + str(int(time.time())) + '.csv', 'w+') as csvfile:
        results_file = csv.writer(csvfile, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        results_file.writerow(['Generation', 'Max Fitness', 'Avg Fitness'])
        while not MAX_COLLISION in [x.fitness() for x in population] and ((generation <= max_generations) if max_generations else True):
            register_loop(population = population,generation = generation,results_file = results_file)
            population = state.generateNextPopulation(listener=listener, population=population, n=POPULATION_SIZE, method=method, elitist=elitist)
            generation += 1

        register_loop(population = population,generation = generation,results_file = results_file)

        for x in population:
            if x.fitness() == MAX_COLLISION:
                print x.stat

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Unknown command-line argument", c)? (<ipython-input-7-9db33b825e7f>, line 94)